In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import pickle
logging.basicConfig(filename='logging.log' , level= logging.INFO, format = '%(levelname)s, %(asctime)s %(message)s'  )


In [84]:
#make connection to the database
try:  
    import mysql.connector as conn
    mydb = conn.connect(user = 'root',host = 'localhost', database = 'test5', passwd= '431170')
    logging.info('Successfully connected the database')
except Exception as e:
    logging.exception(str(e)+ 'connection error')

#make cursor and table in database
try:
    cursor = mydb.cursor()
    cursor.execute('create table test5.class_algeria(day float, month float, humidity float , wind float, rain float, dmc float, dc float, spread float, region float, classes float)')
    mydb.commit()
    logging.info('table created')
except Exception as e:
    logging.exception(str(e )+ 'error occured while new table form')

In [85]:
try:
    data = pd.read_csv('df_final_with10.csv', index_col=0)
    sample_data = data.sample(20)
    #scaling data
    X_columns  = ['Humidity', 'Wind', 'Rain', 'DMC', 'DC', 'Spread']
    X_scale = sample_data[X_columns]
    from sklearn.preprocessing import MinMaxScaler
    Mim_max_X = MinMaxScaler()
    my_data = sample_data.copy()
    X_part = pd.DataFrame(Mim_max_X.fit_transform(X_scale), columns = X_columns, index=sample_data.index)
    my_data.drop(X_columns,axis = 1, inplace = True)
    df_new = my_data.join(X_part)
    col_order = ['Day',  'Month',  'Humidity',  'Wind',  'Rain',  'DMC',    'DC',  'Spread',  'Region',  'Classes']
    df_new = df_new[col_order]
    cursor = mydb.cursor()
    cursor.execute('delete from test5.class_algeria')
    mydb.commit()
    for i, rows in df_new.iterrows():
        row_in = tuple(rows)
        cursor.execute('insert into test5.class_algeria values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s )', row_in)
    mydb.commit()
    logging.info('insertion completed successfully')
except Exception as e:
    logging.exception(str(e), 'error occured while insertion')
    mydb.rollback()


In [109]:
from flask import Flask, request, jsonify
app = Flask(__name__)
import logging 
logging.basicConfig(filename= 'logging.log', level= logging.INFO , message= '%(levelname)s, %(asctime)s, %(message)s')




        
##buraya verileri fetch etmek kaliyor
##sonra verileri x v y olarak ayir ve minmaxtan gecir


In [87]:
@app.route('/mass_class', methods = ['GET', 'POST'])
def fetching_data():
 
    try:
        user = request.json['user']
        host = request.json['host']
        database = request.json['database']
        password = request.json['password']
        import mysql.connector as conn
        mydb = conn.connect(user = user, host = host, database = database, passwd = password)
        logging.info('connection is successfull')
    except Exception as e:
        logging.exception(str(e)+ 'error occured during connection to database')
    try:
        cursor = mydb.cursor()
        cursor.execute('select * from class_algeria')
        fetch_data = [list(i) for i in cursor.fetchall()]
        down_data = pd.DataFrame(fetch_data, columns=('Day',  'Month',  'Humidity',  'Wind',  'Rain',  'DMC',    'DC',  'Spread',  'Region',  'Classes'))    
        X_inserted = down_data.drop('Classes', axis =1)
    except Exception as e:
        logging.exception(str(e)+ 'sth happened while fetching data from database')
    try:
        loaded_class_model = pickle.load(open('class_model.pkl', 'rb'))
        y_pred = loaded_class_model.predict(X_inserted)
        y_modified = ['Fire' if i ==1  else 'Not Fire' for i in y_pred]
        y_modified_s = str(y_modified)
        return jsonify(y_modified_s)
    except Exception as e:
        logging.exception(str(e)+ 'sth happened while predicting the data with model')




In [113]:
if __name__ == '__main__':
    app.run(port = 8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [143]:
from flask import Flask, request, jsonify
app = Flask(__name__)
import logging 
logging.basicConfig(filename= 'logging.log', level= logging.INFO , message= '%(levelname)s, %(asctime)s, %(message)s')

In [63]:
#make connection to the database
try:  
    app = Flask(__name__)
    import mysql.connector as conn
    mydb = conn.connect(user = 'root',host = 'localhost', database = 'test5', passwd= '431170')
    logging.info('Successfully connected the database')
except Exception as e:
    logging.exception(str(e)+ 'connection error')

#make cursor and table in database
try:
    cursor = mydb.cursor()
    cursor.execute('create table test5.regression_algeria(day float, month float, humidity float , wind float, rain float, dmc float, dc float, spread float, region float, classes float)')
    mydb.commit()
    logging.info('table created')
except Exception as e:
    logging.exception(str(e )+ 'error occured while new table form')

In [64]:
try:
    data = pd.read_csv('df_final_with10.csv', index_col=0)
    sample_data = data.sample(20)
    #scaling data
    X_columns  = ['Humidity', 'Wind',  'DMC', 'DC', 'Spread']
    X_scale = sample_data[X_columns]
    from sklearn.preprocessing import MinMaxScaler
    Mim_max_X = MinMaxScaler()
    my_data = sample_data.copy()
    X_part = pd.DataFrame(Mim_max_X.fit_transform(X_scale), columns = X_columns, index=sample_data.index)
    my_data.drop(X_columns,axis = 1, inplace = True)
    df_new = my_data.join(X_part)
    col_order = ['Day',  'Month',  'Humidity',  'Wind',  'Rain',  'DMC',    'DC',  'Spread',  'Region',  'Classes']
    df_new = df_new[col_order]
    cursor = mydb.cursor()
    cursor.execute('delete from test5.regression_algeria')
    mydb.commit()
    for i, rows in df_new.iterrows():
        row_in = tuple(rows)
        cursor.execute('insert into test5.regression_algeria values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s )', row_in)
    mydb.commit()
    logging.info('insertion completed successfully')
except Exception as e:
    logging.exception(str(e), 'error occured while insertion')
    mydb.rollback()


In [65]:
@app.route('/mass_regress', methods = ['GET', 'POST'])
def fetchingreg_data():
    
    try:
        user = request.json['user']
        host = request.json['host']
        database = request.json['database']
        password = request.json['password']
        import mysql.connector as conn
        mydb = conn.connect(user = user, host = host, database = database, passwd = password)
        logging.info('connection is successfull')
    except Exception as e:
        logging.exception(str(e)+ 'error occured during connection to database')
    try:
        cursor = mydb.cursor()
        cursor.execute('select * from regression_algeria')
        fetch_data = [list(i) for i in cursor.fetchall()]
        down_data = pd.DataFrame(fetch_data, columns=('Day',  'Month',  'Humidity',  'Wind',  'Rain',  'DMC',    'DC',  'Spread',  'Region',  'Classes'))    
        X_inserted = down_data.drop('Rain', axis =1)
    except Exception as e:
        logging.exception(str(e)+ 'sth happened while fetching data from database')
    try:
        loaded_class_model = pickle.load(open('regression_model.pkl', 'rb'))
        y_pred = loaded_class_model.predict(X_inserted)
        #Mim_max_X.inverse_transform(y_pred.reshape(-1, 1))
        y_modified_s = str(np.round(y_pred,2))
        return jsonify(y_modified_s)

    except Exception as e:
        logging.exception(str(e)+ 'sth happened while predicting the data with model')



In [111]:
if __name__ == '__main__':
    app.run(port = 8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [229]:
from flask import Flask, request, jsonify
app = Flask(__name__)
import logging 
logging.basicConfig(filename= 'logging.log', level= logging.INFO , message= '%(levelname)s, %(asctime)s, %(message)s')

In [230]:
@app.route('/single_regress', methods = ['POST'])
def single_regression():
    try:
        data=request.json['data']
        print(data)
        new_data=[list(data.values())]
    
    except Exception as e:
        logging.exception(str(e)+ 'error occured whıle fetchıng data from postman')

    try:
        loaded_class_model = pickle.load(open('regression_model.pkl', 'rb'))
        y_pred = loaded_class_model.predict(new_data)[0]
        y_modified_s = str(y_pred)
        return jsonify(y_modified_s)
    except Exception as e:
        logging.exception(str(e)+ 'sth happened while predicting the data with model')

    

In [231]:
if __name__ == '__main__':
    app.run(port = 8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
{'Day': 7, 'Month': 10, 'Humidity': 1, 'Wind': 1, 'DMC': 1, 'DC': 1, 'Spread': 0, 'Region': 0, 'Classes': 1}


In [232]:
from flask import Flask, request, jsonify
app = Flask(__name__)
import logging 
logging.basicConfig(filename= 'logging.log', level= logging.INFO , message= '%(levelname)s, %(asctime)s, %(message)s')

In [233]:
@app.route('/single_class', methods = ['POST'])
def single_classif():
    try:
        data=request.json['data']
        print(data)
        new_data=[list(data.values())]
    
    except Exception as e:
        logging.exception(str(e)+ 'error occured whıle fetchıng data from postman')

    try:
        loaded_class_model = pickle.load(open('class_model.pkl', 'rb'))
        y_pred = loaded_class_model.predict(new_data)[0]
        y_str = str(y_pred)
        return jsonify(y_str)
    except Exception as e:
        logging.exception(str(e)+ 'sth happened while predicting the data with model')

In [234]:
if __name__ == '__main__':
    app.run(port = 8000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
{'Day': 7, 'Month': 10, 'Humidity': 1, 'Wind': 1, 'Rain': 0.4, 'DMC': 1, 'DC': 1, 'Spread': 0, 'Region': 0}
